In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open('output2.json') as json_data:
    output = json.load(json_data)

In [3]:
df_output=pd.DataFrame.from_dict(output)

In [4]:
df_output.sort_values(by='timestamp', inplace=True)
str_cols=["eventType","uid","impressionId"]
for col in str_cols:
    df_output[col]=df_output[col].astype(str)
   


In [5]:
df_output.head()

,eventType,uid,timestamp,ip,impressionId
0,display,6a70dfc6-c32a-4ac2-9aa9-333ba8b92f23,1589157579,152.123.63.26,1bf5b860-f9fa-481d-9b89-60f2d278bb71
11,display,ba9dc386-4761-47ab-b3cd-aff0c51169b3,1589157581,32.134.155.53,7f410710-42ef-4b12-860a-ceea45a3034a
10,display,d538463c-fb0d-4b9b-9e64-4f74f483e5e2,1589157581,120.37.17.159,7c0dff2b-23c5-424b-b50d-055c5957acb0
8,display,e306fae1-8655-4b7b-94ff-41ebe09262d2,1589157581,163.135.203.240,bdda29f2-bd52-4512-9ad0-5d0a8cc88db1
7,display,ec9e211a-3a53-411a-8abf-bd31ffdc87b6,1589157581,197.40.27.215,de5f3557-5d5d-4b8c-988f-61d6c52539b9


In [6]:
df_output.shape

(1986, 5)

# Données
The data correspond to clicks and display events of advertising banners on a website  
These data were read in streaming from Kafka, we stored 2 min of stream in order to analyze them.  
The purpose of these analyzes is to detect fraudulent click activity on advertisements.  


The generated variables are:  

__EventType__ =which gives us the type of the event display or click.  
__uid__  = user Id which associates an Id to a user by browser.    
__timestamp__ = The time when the event (click or display) happened on the website  
__ip__ = is the ip address that generated the event  
__impressionId__= identifier of the advertising display



In [7]:
from datetime import datetime

timestamp_1 = int(df_output.iloc[0,2])
timestamp_2 = int(df_output.iloc[-1,2])
firstTimestamp = datetime.fromtimestamp(timestamp_1)
lastTimestamp= datetime.fromtimestamp(timestamp_2)


print("First timestamp =", firstTimestamp)
print("Last timestamp =", lastTimestamp)

timestamp_2-timestamp_1

First timestamp = 2020-05-11 02:39:39
Last timestamp = 2020-05-11 02:41:29


110

The data corresponds to 110 seconds of streaming events

# Data analysis:

## <font color='red'> Ip adress VS uid </font>
here we want to know how many user ids correspond to a unique IP 

In [8]:
Ip_uid=df_output[["ip","uid"]].groupby("ip").count()

In [9]:
Ip_uid=df_output[["ip","uid"]].groupby("ip").count().sort_values(by="uid", ascending=False)

In [10]:
Ip_uid.head(2)

,uid
ip,
238.186.83.58,219
110.50.254.26,6


In [11]:
Ip_uid.describe()

,uid
count,1321.000000
mean,1.503407
std,6.015651
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,219.000000


__With 2 min of stream, 75% of ip addresses have 2 different user IDs, which can be considered as normal because the user ID changes if you change the browser. But a browser change more than 2 times in two minutes is suspicious and therefore wa can consider it as suspicious__

In [12]:
print("number of fraudulent events via a suspicious Uid detection: " , len(Ip_uid[Ip_uid["uid"]>2]))

number of fraudulent events via a suspicious Uid detection:  42


## <font color='red'> Uid vs ip Adress </font>
here we want to know how many IPs correspond to a unique UID

In [13]:
uid_ip=df_output[["uid","ip"]].groupby("uid").count()

In [14]:
uid_ip=df_output[["uid","ip"]].groupby("uid").count().sort_values(by="ip", ascending=False)

In [15]:
uid_ip.head()

,ip
uid,
f978c855-1fd0-4bf3-88a6-9d8dcbdbb7e0,32
ef74484e-b906-430a-a50a-87c2cba511cb,29
8b14b017-31de-46d3-8d0c-b7b04b78de9b,27
6106c6b7-31f4-4647-a777-e1a529b79bc4,27
3efe8e27-6f57-4428-81db-ed88e2e67279,26


In [16]:
uid_ip.describe()

,ip
count,875.000000
mean,2.269714
std,4.568866
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,32.000000


__with stream of 2min, 75% of user IDs have a single IP address, a user ID that has more than one IP address is supicious. We choose to consider this pattern as fraudulent__

In [17]:
print("number of fraudulent events via a suspicious Uid detection: " , len(uid_ip[uid_ip["ip"]>1]))

number of fraudulent events via a suspicious Uid detection:  202


##  <font color='red'> CTR : Click Through Rate </font>

In [18]:
#split of the dataframe by clicks and displays

df_clicks = df_output[df_output['eventType']=="click"]
df_clicks["clicks_count"]=1
df_clicks.head(2)

C:\Users\33664\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,eventType,uid,timestamp,ip,impressionId,clicks_count
56,click,a7c33f8c-6d4a-440f-aa0c-7c2ae31a443d18,1589157584,238.186.83.58,4690514b-238c-4ce5-a828-ef6041d76956,1
58,click,2d265efd-98a0-4f52-918d-67371a38b58e19,1589157584,238.186.83.58,c1b24a1f-f743-4a33-99db-54ebd9ee898a,1


In [19]:
df_clicks.shape

(446, 6)

In [20]:
df_displays = df_output[df_output['eventType']=="display"]
df_displays["displays_count"]=1
df_displays.shape

C:\Users\33664\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(1540, 6)

###  <font color='green'> CTR per User ID</font>

clicks_count/displays_count per uid  

We choose to calculate the CTR per Uid because we have less fraud via uid than Ip adress
If CTR> 50%, ie if a customer clicks more than once on two times on an ad, the corresponding events are considered as suspicious and we want to detect this pattern
In order to not include people who had one display and one click which is not necessarily suspicious, we remove events with exactly 1 click and 1 display

In [21]:
df_CTR_ID=  pd.merge(df_displays[["uid","displays_count"]].groupby("uid").count(),df_clicks[["uid", "clicks_count"]].groupby("uid").count(), on=["uid"], how="outer")

In [22]:
df_CTR_ID.shape

(875, 2)

In [23]:
df_CTR_ID.displays_count.value_counts()

1.0     823
15.0      8
11.0      6
18.0      5
14.0      5
13.0      5
9.0       4
17.0      4
12.0      4
16.0      4
21.0      2
19.0      2
8.0       1
2.0       1
Name: displays_count, dtype: int64

In [24]:
df_CTR_ID.fillna(0,inplace=True)

In [25]:
df_CTR_ID.clicks_count.value_counts()

0.0     674
1.0     152
6.0      10
7.0       9
4.0       9
5.0       7
8.0       6
3.0       3
11.0      2
10.0      1
9.0       1
2.0       1
Name: clicks_count, dtype: int64

In [26]:
df_CTR_ID=df_CTR_ID.assign(CTR = df_CTR_ID['clicks_count']/ df_CTR_ID['displays_count'])

In [27]:
df_CTR_ID.describe()

,displays_count,clicks_count,CTR
count,875.000000,875.000000,875.0
mean,1.760000,0.509714,inf
std,3.179525,1.467679,NaN
min,0.000000,0.000000,0.0
25%,1.000000,0.000000,0.0
50%,1.000000,0.000000,0.0
75%,1.000000,0.000000,0.0
max,21.000000,11.000000,inf


__there is a uid with a click and without display, but this is due to a delay in receiving the corresponding display event we delete this uid from the data__  
__We also delete the uids which have 1 click and 1 display__

In [28]:
df_output[df_output['uid']=='6a0f34ea-2700-4ca2-b21b-ba315f5d5f8e19']

,eventType,uid,timestamp,ip,impressionId
1893,click,6a0f34ea-2700-4ca2-b21b-ba315f5d5f8e19,1589157684,238.186.83.58,c6fd074f-4704-41ee-bedd-b347c8cae21d


In [29]:
print(datetime.fromtimestamp(1589157684))

2020-05-11 02:41:24


In [30]:
df_CTR_ID.drop(index="6a0f34ea-2700-4ca2-b21b-ba315f5d5f8e19", axis=0, inplace=True)

In [31]:
index=df_CTR_ID[(df_CTR_ID["displays_count"]==1)&(df_CTR_ID['clicks_count']==1)].index
df_CTR_ID.drop(index=index, axis=0,inplace=True)

In [32]:
print("number of events with suspicious CTR : " , len(df_CTR_ID[df_CTR_ID["CTR"]>0.5]))

number of events with suspicious CTR :  11
